## Fix missing node labels from all node labels files

In [1]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
from tqdm import tqdm
import json

In [2]:
import hashlib

In [3]:
import pickle
import pandas as pd
import numpy as np

In [4]:
KG_PATH = '../resources/knowledge_graphs/'
NodeLabelsFile = KG_PATH + 'nodeLabels_v1.0.pickle'

In [5]:
with open(NodeLabelsFile, 'rb') as filep:
    nodeLabels = pickle.load(filep)

In [6]:
len(nodeLabels)

753369

In [7]:
#missing labels for nodes = 4457
with open('../resources/NPKG_missing_nodelabels.txt') as filei:
    missing_labels = filei.readlines()
len(missing_labels)

4457

In [8]:
nodeLabelsPatch = {}

In [10]:
#check in pheknowlator
PL_PATH = 'PheKnowLatorv3/'
nodesplfile = PL_PATH + 'resources/knowledge_graphs/PheKnowLator_v3.0.0_full_instance_inverseRelations_OWLNETS_NodeLabels.txt'

In [11]:
nodespl = pd.read_csv(nodesplfile, sep='\t')

In [12]:
nodespl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743399 entries, 0 to 743398
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   entity_type             743285 non-null  object
 1   integer_id              743367 non-null  object
 2   entity_uri              743317 non-null  object
 3   label                   743214 non-null  object
 4   description/definition  743208 non-null  object
 5   synonym                 743157 non-null  object
dtypes: object(6)
memory usage: 34.0+ MB


In [21]:
nodespllist = nodespl.entity_uri.tolist()
len(nodespllist)

743399

In [24]:
found_in_pl = []
still_missing = []
for item in missing_labels:
    entity = item.strip()
    if '<'+entity+'>' in nodespllist:
        found_in_pl.append(entity)
    else:
        still_missing.append(entity)

In [26]:
len(found_in_pl)

4429

In [36]:
for item in found_in_pl:
    entity = '<'+item+'>'
    label = nodespl.loc[nodespl['entity_uri'] == entity]['label']
    if not label.empty:
        nodeLabelsPatch[item] = label.values[0]
        nodeLabels[item] = label.values[0]
    else:
        still_missing.append(item)
len(nodeLabelsPatch)

4429

In [37]:
len(still_missing)

28

In [30]:
###add the missing ones manually
##not available N=9
still_missing

['http://purl.obolibrary.org/obo/CHEBI_230539',
 'http://purl.obolibrary.org/obo/CHEBI_14581053',
 'http://purl.obolibrary.org/obo/CHEBI_172345',
 'http://purl.obolibrary.org/obo/CHEBI_172341',
 'http://purl.obolibrary.org/obo/CHEBI_168569',
 'http://purl.obolibrary.org/obo/CHEBI_3855995',
 'http://purl.obolibrary.org/obo/CHEBI_3855960',
 'http://purl.obolibrary.org/obo/CHEBI_385590',
 'http://purl.obolibrary.org/obo/CHEBI_385594',
 'http://purl.obolibrary.org/obo/SLC47A1',
 'http://purl.obolibrary.org/obo/CHEBI_273632',
 'http://purl.obolibrary.org/obo/CHEBI_6416',
 'http://purl.obolibrary.org/obo/CHEBI_1757421',
 'http://purl.obolibrary.org/obo/http_//www.ebi.ac.uk/cellline',
 'https://www.ncbi.nlm.nih.gov/snp/rs369019849',
 'https://www.ncbi.nlm.nih.gov/snp/rs756952607',
 'https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000651398',
 'https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000564319',
 'http://purl.obolibrary.org/obo/CHEBI_35373',
 'http

In [55]:
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_230539'] = 'NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_14581053'] = 'NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_172345'] ='chaetoglobosin P'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_172341']='Luotonin A'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_168569']='Cadabicine methyl ether'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_3855995']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_3855960']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_385590']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_385594']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/SLC47A1']='SLC47A1'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_273632']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_6416']='leucocyanidin'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_1757421']='NaN'
nodeLabelsPatch['http://purl.obolibrary.org/obo/http_//www.ebi.ac.uk/cellline']='NaN'
nodeLabelsPatch['https://www.ncbi.nlm.nih.gov/snp/rs369019849']='rs369019849'
nodeLabelsPatch['https://www.ncbi.nlm.nih.gov/snp/rs756952607']='rs756952607'
nodeLabelsPatch['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000651398']='family with sequence similarity 153 member C, pseudogene'
nodeLabelsPatch['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000564319']='MTOR associated protein, LST8 homolog'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_35373']='D-glucosaminide'
nodeLabelsPatch['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000510441']='exocyst complex component 3'
nodeLabelsPatch['http://purl.obolibrary.org/obo/GO_0070871']='cell wall organization involved in conjugation with cellular fusion'
nodeLabelsPatch['http://purl.obolibrary.org/obo/PR_000050360']='macro CK type 2 (human)'
nodeLabelsPatch['http://purl.obolibrary.org/obo/PR_000048538']='hNDUFA11/InitMet'
nodeLabelsPatch['http://purl.obolibrary.org/obo/CHEBI_81152']='Amaroswerin'
nodeLabelsPatch['https://reactome.org/content/detail/R-HSA-5693540']='MRN activates ATM'
nodeLabelsPatch['http://purl.obolibrary.org/obo/SO_0001885']='TFBS_translocation'
nodeLabelsPatch['http://purl.obolibrary.org/obo/UBERON_4300184']='neural spine 5'
nodeLabelsPatch['http://purl.obolibrary.org/obo/OAE_0004475']='acute prerenal failure AE'

In [56]:
len(nodeLabelsPatch)

4457

In [57]:
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_230539'] = 'NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_14581053'] = 'NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_172345'] ='chaetoglobosin P'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_172341']='Luotonin A'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_168569']='Cadabicine methyl ether'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_3855995']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_3855960']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_385590']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_385594']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/SLC47A1']='SLC47A1'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_273632']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_6416']='leucocyanidin'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_1757421']='NaN'
nodeLabels['http://purl.obolibrary.org/obo/http_//www.ebi.ac.uk/cellline']='NaN'
nodeLabels['https://www.ncbi.nlm.nih.gov/snp/rs369019849']='rs369019849'
nodeLabels['https://www.ncbi.nlm.nih.gov/snp/rs756952607']='rs756952607'
nodeLabels['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000651398']='family with sequence similarity 153 member C, pseudogene'
nodeLabels['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000564319']='MTOR associated protein, LST8 homolog'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_35373']='D-glucosaminide'
nodeLabels['https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=ENST00000510441']='exocyst complex component 3'
nodeLabels['http://purl.obolibrary.org/obo/GO_0070871']='cell wall organization involved in conjugation with cellular fusion'
nodeLabels['http://purl.obolibrary.org/obo/PR_000050360']='macro CK type 2 (human)'
nodeLabels['http://purl.obolibrary.org/obo/PR_000048538']='hNDUFA11/InitMet'
nodeLabels['http://purl.obolibrary.org/obo/CHEBI_81152']='Amaroswerin'
nodeLabels['https://reactome.org/content/detail/R-HSA-5693540']='MRN activates ATM'
nodeLabels['http://purl.obolibrary.org/obo/SO_0001885']='TFBS_translocation'
nodeLabels['http://purl.obolibrary.org/obo/UBERON_4300184']='neural spine 5'
nodeLabels['http://purl.obolibrary.org/obo/OAE_0004475']='acute prerenal failure AE'

In [58]:
len(nodeLabels)

757826

In [60]:
with open(KG_PATH+'nodeLabels_v1.0.1.pickle', 'wb') as fileo:
    pickle.dump(nodeLabels, fileo)